In [ ]:
import numpy as np
from scipy.fft import fft, fftfreq
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.ticker import FixedLocator, NullLocator, FixedFormatter
fontsize = 9
lw = 0.75
matplotlib.rc('font', **{'family': 'Arial', 'size': fontsize})
matplotlib.rc('axes', **{'linewidth': 0.75, 'labelsize': fontsize})
matplotlib.rc('xtick', **{'labelsize': fontsize})
matplotlib.rc('ytick', **{'labelsize': fontsize})
matplotlib.rc('xtick.major', **{'width': lw, 'size':3})
matplotlib.rc('ytick.major', **{'width': lw, 'size':3})
matplotlib.rc('ytick.minor', **{'width': lw, 'size':1.5})

In [ ]:
blobs = [#np.load('../Sardinia_tran_default.npz', allow_pickle=True),
         np.load('../Sardinia_tran_FSACTI0201GGR3_double_H.npz', allow_pickle=True)]
data = [blob['data'].item() for blob in blobs]
time = [blob['time'] for blob in blobs]

In [ ]:
Vr = [d['bus']['m:ur'] for d in data]
Vi = [d['bus']['m:ui'] for d in data]
V = [np.sqrt(vr**2 + vi**2) for vr,vi in zip(Vr,Vi)]
Vnorm = []
for v in V:
    n_samples = v.shape[0]
    m = np.tile(v.mean(axis=0), (n_samples, 1))
    s = np.tile(v.std(axis=0), (n_samples, 1))
    Vnorm.append((v - m) / s)
speed = [d['gen']['s:xspeed'] for d in data]

In [ ]:
H = [blob['inertia'].item() for blob in blobs]
E = [blob['energy'].item() for blob in blobs]
M = [blob['momentum'].item() for blob in blobs]

In [ ]:
speedf, Vf = [], []
dur = 60 # [s]
dt = time[0][1] - time[0][0]
n_samples = int(dur/dt)
freq = fftfreq(n_samples, dt)[:n_samples//2]
for spd,v in zip(speed, Vnorm):
    speedf.append(np.zeros((freq.size, spd.shape[1])))
    for j in range(spd.shape[1]):
        n_trials = spd.shape[0] // n_samples
        X = np.reshape(spd[:n_trials*n_samples,j], (n_trials, n_samples))
        tmp = fft(X-1)
        speedf[-1][:,j] = np.mean(2.0 / n_samples * np.abs(tmp[:,:n_samples//2]), axis=0)

    Vf.append(np.zeros((freq.size, v.shape[1])))
    for j in range(v.shape[1]):
        n_trials = v.shape[0] // n_samples
        X = np.reshape(v[:n_trials*n_samples,j], (n_trials, n_samples))
        tmp = fft(X)
        Vf[-1][:,j] = np.mean(2.0 / n_samples * np.abs(tmp[:,:n_samples//2]), axis=0)

In [ ]:
fig,ax = plt.subplots(2, 1, sharex=True, figsize=(5,4))
j = 2
col = 'krg'
lw = 0.5
for i,(t,spd,v) in enumerate(zip(time, speed, Vnorm)):
    ax[0].plot(t, spd[:,j], col[i], lw=lw, label='M = {:.0f} MJs'.format(M[i]))
    ax[1].plot(t, v[:,j], col[i], lw=lw)
ax[0].legend(loc='best', fontsize=8, frameon=False)
ax[0].set_xlim([0, 120])
ax[-1].set_xlabel('Time [s]')
ax[0].set_ylabel('ω [p.u.]')
ax[1].set_ylabel('V norm.')
sns.despine()
fig.tight_layout()
plt.savefig('omega_V_norm_stoch_load.pdf')

In [ ]:
n_bins = 101
edges = np.linspace(-5, 5, n_bins)
N = [np.zeros((n_bins-1, v.shape[1])) for v in Vnorm]
for i,v in enumerate(Vnorm):
    for j in range(v.shape[1]):
        N[i][:,j],_ = np.histogram(v[:,j], bins=edges, density=True)

In [ ]:
rows,cols = 3,6
device_names = blobs[0]['device_names'].item()['bus']
fig,ax = plt.subplots(rows, cols, figsize=(cols*1.5, rows*1.5), sharex=True, sharey=True)
for n,blob in enumerate(blobs):
    names = blob['device_names'].item()['bus']
    for k,name in enumerate(device_names[:rows*cols]):
        i,j = k//cols, k%cols
        try:
            idx = names.index(name)
            ax[i,j].plot(edges[:-1], N[n][:,idx], col[n], lw=1)
            ax[i,j].set_title(name.split('___')[0], fontsize=8)
        except:
            print(f'Device name {name} missing')
for a in ax[-1,:]:
    a.set_xlabel('V norm.')
for a in ax[:,0]:
    a.set_ylabel('PDF')
sns.despine()
fig.tight_layout()
plt.savefig('V_norm_distr_stoch_load.pdf')

In [ ]:
rows,cols = 3,6
device_names = blobs[0]['device_names'].item()['bus']
fig,ax = plt.subplots(rows, cols, figsize=(cols*1.5, rows*1.5), sharex=True, sharey=True)
ticks = np.array([0.01, 0.1, 1, 10, 100])
for n,blob in enumerate(blobs):
    names = blob['device_names'].item()['bus']
    for k,name in enumerate(device_names[:rows*cols]):
        i,j = k//cols, k%cols
        try:
            idx = names.index(name)
            ax[i,j].semilogx(freq, 20*np.log10(Vf[n][:,idx]), col[n], lw=0.75, label='M = {:.0f} MJs'.format(M[n]))
            ax[i,j].set_title(name.split('___')[0], fontsize=8)
            ax[i,j].xaxis.set_major_locator(FixedLocator(ticks))
            ax[i,j].xaxis.set_minor_locator(NullLocator())
            ax[i,j].xaxis.set_major_formatter(FixedFormatter([f'{tick:g}' for tick in ticks]))
            ax[i,j].set_xlim(ticks[[0,-1]])
        except:
            print(f'Device name {name} missing')
ax[0,0].legend(loc='lower left', frameon=False, fontsize=6)
for a in ax[-1,:]:
    a.set_xlabel('Frequency [Hz]')
for a in ax[:,0]:
    a.set_ylabel('[dB20]')
sns.despine()
fig.tight_layout()
plt.savefig('V_norm_spectra_stoch_load.pdf')

In [ ]:
rows,cols = 3,6
device_names = blobs[0]['device_names'].item()['bus']
fig,ax = plt.subplots(rows, cols, figsize=(cols*1.5, rows*1.5), sharex=True, sharey=True)
ticks = np.array([0.01, 0.1, 1, 10, 100])
for n,blob in enumerate(blobs):
    names = blob['device_names'].item()['bus']
    for k,name in enumerate(device_names[:rows*cols]):
        i,j = k//cols, k%cols
        try:
            idx = names.index(name)
            ax[i,j].semilogx(freq, 10*np.log10(speedf[n][:,idx]), col[n], lw=0.75, label='M = {:.0f} MJs'.format(M[n]))
            ax[i,j].set_title(name.split('___')[0], fontsize=8)
            ax[i,j].xaxis.set_major_locator(FixedLocator(ticks))
            ax[i,j].xaxis.set_minor_locator(NullLocator())
            ax[i,j].xaxis.set_major_formatter(FixedFormatter([f'{tick:g}' for tick in ticks]))
            ax[i,j].set_xlim(ticks[[0,-1]])
        except:
            print(f'Device name {name} missing')
ax[0,0].legend(loc='lower left', frameon=False, fontsize=6)
for a in ax[-1,:]:
    a.set_xlabel('Frequency [Hz]')
for a in ax[:,0]:
    a.set_ylabel('[dB20]')
sns.despine()
fig.tight_layout()
plt.savefig('speed_spectra_stoch_load.pdf')

In [ ]:
raise Exception('stop here')

In [ ]:
TF_data = np.load('../TF_-6.0_2.0_100.npz', allow_pickle=True)

In [ ]:
TF_data.files

In [ ]:
TF_data['mag'].shape

In [ ]:
blobs[0]['device_names'].item()['gen']

In [ ]:
rows,cols = 3,6
w,h = 2,1.5
device_names = blobs[0]['device_names'].item()['gen']
fig,ax = plt.subplots(rows, cols, figsize=(cols*w, rows*h), sharex=True, sharey=True)
ticks = np.array([0.001, 0.01, 0.1, 1, 10, 100])
F0 = ticks[1]
for n,blob in enumerate(blobs):
    names = blob['device_names'].item()['gen']
    for k,name in enumerate(device_names[:rows*cols]):
        i,j = k//cols, k%cols
        try:
            idx = names.index(name)
            jdx = np.abs(freq-F0).argmin()
            y = 20*np.log10(speedf[n][:,idx])
            ax[i,j].semilogx(freq, y-y[jdx], col[n], lw=0.75, label='M = {:.0f} MJs'.format(M[n]))
            idx = [sm_name in name for sm_name in TF_data['SM_names'][0]].index(True)
            jdx = np.abs(TF_data['F']-F0).argmin()
            y = TF_data['mag'][1,:,idx]
            ax[i,j].semilogx(TF_data['F'], y-y[jdx], col[n]+'--', lw=0.75)
            ax[i,j].set_title(name.split('___')[0], fontsize=8)
            ax[i,j].xaxis.set_major_locator(FixedLocator(ticks))
            ax[i,j].xaxis.set_minor_locator(NullLocator())
            ax[i,j].xaxis.set_major_formatter(FixedFormatter([f'{tick:g}' for tick in ticks]))
            ax[i,j].set_xlim(ticks[[0,-1]])
        except:
            print(f'Device name {name} missing')
ax[0,0].legend(loc='lower left', frameon=False, fontsize=6)
for a in ax[-1,:]:
    a.set_xlabel('Frequency [Hz]')
for a in ax[:,0]:
    a.set_ylabel('[dB20]')
sns.despine()
fig.tight_layout()
plt.savefig('speed_spectra_stoch_load.pdf')